# NUV-10 fs Time-Resolved Spectroscopy Results (2022 / 04 / 15) 

In [1]:
import numpy as np
import pandas as pd

import scipy as sp
from scipy import signal
from scipy.constants import h, c, e
from scipy.fft import fft, fft2, fftfreq, fftshift

import matplotlib as mp
from matplotlib import cm
from matplotlib import rc
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
# import matplotlib_latex_bridge as mlb
import matplotlib.pyplot as plt
%matplotlib inline

# import math
# import tkinter as tk
# from tkinter import filedialog

mp.rcParams.update({"text.usetex":False})
mp.rcParams["mathtext.fontset"] = "cm"
mp.rcParams["mathtext.rm"] = "Times New Roman"
mp.rcParams["mathtext.it"] = "Times New Roman:italic"
mp.rcParams["mathtext.bf"] = "Times New Roman:bold"
rc("font", family="Times New Roman")

## Measurment information
Sample: DMSO.  
Scan: from -300 fs to 1800 fs.  
Scan step: 1 fs for 2000 pts.  
Scan loops: 5 loops for average. 

## Data import

### 1. dT and T import

In [2]:
class DataImport:
    def __init__(self):
        return None

    def result_import(self, dT_file_path:str, data_name:str, transpose:bool=True, reverse:bool=False):
        self.result_import = pd.read_table(
            dT_file_path, sep="\t", header=None,
            dtype={"Name": str, "Value": "double"},
            encoding="big5", float_precision="high"
        )

        result0 = self.result_import.transpose() if transpose else self.result_import
        result = result0.iloc[::-1].reset_index(drop=True) if reverse else result0
        
        print(f"Imported {data_name} path:\n{dT_file_path}")
        return result
    
    
    def mla_import(self, T_file_path:str):
        print(f"Imported T path:\n{T_file_path}")
        self.mla_import = pd.read_table(
            T_file_path, sep="\t", header=3,
            dtype={"Name": str, "Value": "double"}, encoding="big5"
        )
        return self.mla_import

In [3]:
dT_b = DataImport()
dT_b.result_import(dT_file_path="./Examples/dT  -200.10 1.00 2000.00   5-ft scan-b",
                   data_name="dT backward scan", transpose=True, reverse=True)

Imported dT backward scan path:
./Examples/dT  -200.10 1.00 2000.00   5-ft scan-b


,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-0.000021,-0.000055,-0.000046,-0.000052,-0.000043,-0.000023,-1.320000e-05,-0.000026,-0.000060,-0.000031,...,-0.000243,-0.000245,-0.000287,-0.000204,-0.000156,-0.000137,-0.000156,-0.000179,-0.000267,-0.000091
1,-0.000021,-0.000058,-0.000049,-0.000052,-0.000044,-0.000021,-9.200000e-06,-0.000027,-0.000052,-0.000020,...,-0.000210,-0.000226,-0.000323,-0.000260,-0.000187,-0.000126,-0.000133,-0.000167,-0.000299,-0.000133
2,-0.000024,-0.000058,-0.000052,-0.000053,-0.000043,-0.000016,-4.400000e-06,-0.000026,-0.000045,-0.000010,...,-0.000191,-0.000209,-0.000348,-0.000328,-0.000247,-0.000136,-0.000119,-0.000147,-0.000310,-0.000184
3,-0.000025,-0.000057,-0.000053,-0.000057,-0.000043,-0.000017,-7.800000e-06,-0.000026,-0.000047,-0.000011,...,-0.000187,-0.000194,-0.000359,-0.000388,-0.000305,-0.000150,-0.000105,-0.000122,-0.000294,-0.000217
4,-0.000023,-0.000051,-0.000051,-0.000055,-0.000040,-0.000016,-9.800000e-06,-0.000028,-0.000044,-0.000013,...,-0.000177,-0.000165,-0.000354,-0.000430,-0.000353,-0.000163,-0.000088,-0.000092,-0.000247,-0.000219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,-0.000026,-0.000036,-0.000024,-0.000026,-0.000025,-0.000026,-2.020000e-05,-0.000028,-0.000026,-0.000023,...,-0.000197,-0.000166,-0.000209,-0.000221,-0.000222,-0.000171,-0.000160,-0.000189,-0.000365,-0.000202
1997,-0.000023,-0.000027,-0.000016,-0.000019,-0.000014,-0.000015,-1.340000e-05,-0.000014,-0.000018,-0.000017,...,-0.000201,-0.000167,-0.000218,-0.000229,-0.000231,-0.000178,-0.000166,-0.000198,-0.000385,-0.000213
1998,-0.000018,-0.000013,-0.000007,-0.000005,-0.000002,0.000001,2.000000e-07,0.000001,-0.000004,-0.000007,...,-0.000214,-0.000189,-0.000241,-0.000249,-0.000251,-0.000193,-0.000181,-0.000212,-0.000429,-0.000242
1999,-0.000016,-0.000012,-0.000007,-0.000005,-0.000003,-0.000002,-6.000000e-07,-0.000004,-0.000008,-0.000008,...,-0.000222,-0.000204,-0.000248,-0.000255,-0.000256,-0.000198,-0.000184,-0.000215,-0.000440,-0.000250


In [4]:
dT_f = DataImport()
dT_f.result_import(dT_file_path="./Examples/dT  -200.10 1.00 2000.00   5-ft scan-f",
                   data_name="dT forward scan", transpose=True, reverse=False)

Imported dT forward scan path:
./Examples/dT  -200.10 1.00 2000.00   5-ft scan-f


,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-0.000016,-0.000042,-0.000037,-0.000019,-3.000000e-05,-0.000024,-0.000017,-1.560000e-05,-0.000033,-0.000016,...,-0.000256,-0.000229,-0.000211,-0.000137,-0.000180,-0.000199,-0.000204,-0.000163,-0.000146,-0.000079
1,-0.000018,-0.000046,-0.000038,-0.000020,-3.080000e-05,-0.000025,-0.000023,-1.960000e-05,-0.000038,-0.000017,...,-0.000256,-0.000225,-0.000214,-0.000141,-0.000184,-0.000203,-0.000207,-0.000163,-0.000159,-0.000082
2,-0.000015,-0.000042,-0.000033,-0.000017,-2.460000e-05,-0.000021,-0.000016,-2.340000e-05,-0.000038,-0.000011,...,-0.000255,-0.000219,-0.000214,-0.000141,-0.000183,-0.000204,-0.000203,-0.000159,-0.000161,-0.000082
3,-0.000015,-0.000039,-0.000031,-0.000018,-2.280000e-05,-0.000020,-0.000008,-1.480000e-05,-0.000035,-0.000005,...,-0.000255,-0.000219,-0.000221,-0.000149,-0.000190,-0.000201,-0.000196,-0.000162,-0.000172,-0.000086
4,-0.000017,-0.000040,-0.000029,-0.000020,-2.200000e-05,-0.000017,-0.000004,-2.000000e-07,-0.000029,-0.000002,...,-0.000249,-0.000220,-0.000229,-0.000160,-0.000189,-0.000196,-0.000193,-0.000173,-0.000191,-0.000087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,-0.000007,-0.000006,-0.000004,-0.000002,-4.000000e-07,0.000004,0.000002,5.000000e-06,0.000005,0.000006,...,-0.000278,-0.000213,-0.000257,-0.000217,-0.000181,-0.000098,-0.000078,-0.000082,-0.000167,-0.000113
1997,-0.000010,-0.000015,-0.000009,-0.000011,-1.080000e-05,-0.000010,-0.000010,-4.200000e-06,-0.000006,-0.000007,...,-0.000270,-0.000197,-0.000237,-0.000196,-0.000164,-0.000089,-0.000068,-0.000066,-0.000142,-0.000102
1998,-0.000026,-0.000041,-0.000028,-0.000033,-2.820000e-05,-0.000030,-0.000028,-2.160000e-05,-0.000020,-0.000020,...,-0.000243,-0.000169,-0.000204,-0.000168,-0.000139,-0.000076,-0.000055,-0.000054,-0.000122,-0.000097
1999,-0.000035,-0.000050,-0.000038,-0.000039,-3.160000e-05,-0.000034,-0.000029,-2.520000e-05,-0.000020,-0.000018,...,-0.000238,-0.000170,-0.000197,-0.000162,-0.000131,-0.000073,-0.000059,-0.000062,-0.000145,-0.000115


In [5]:
T = DataImport()
T.mla_import(T_file_path="./Examples/T@390 mn 100 mv td= 100ms +45 -45 dregee.mld")

Imported T path:
./Examples/T@390 mn 100 mv td= 100ms +45 -45 dregee.mld


,X1,Y1,Unnamed: 2
0,0.01888,0.00058,NaN
1,0.02482,0.00109,NaN
2,0.01897,0.00063,NaN
3,0.01826,0.00057,NaN
4,0.01651,0.00044,NaN
...,...,...,...
91,0.02662,0.00033,NaN
92,0.02383,0.00058,NaN
93,0.02496,0.00066,NaN
94,0.05231,0.00084,NaN


### 2. Wavelength and time calibration

In [ ]:
# time array
time_array = pd.Series(np.linspace(-200,1800,2001))
# wavlength calibration
calibration_reference = pd.read_table("/Users/kentko/Library/Mobile Documents/com~apple~CloudDocs/研究資料/實驗數據/multi channel lockin amp calibration/Wavelength ref of multi ch lock in-96 ch.dat",
                                    sep="\t",
                                    header=0,
                                    dtype={'Name': str, 'Value': float})
wavelength = pd.Series(calibration_reference.loc[:,"390"]).transpose()
print("Time array and wavelength imported")

## Data processing

In [ ]:
def bg(input_data, mean_pts):
    """bg: remove the back ground noise of one dimensional array(onedim_data):
        select the 0 to "mean_pts" elements of 1D-array and take the average them to get  background signmal(bg),
        them return the reult of 1D-array-bg""" 
    # get the average value of selected range
    bg_signal = np.mean(input_data[0:mean_pts])                                                                
    # remove the background 
    bg_result = input_data-bg_signal                                                                            
    return bg_result


def smooth_filter(input_data,t_step,smooth_pts,filtertype):
    """smooth_hass: smooth the input data by take average of each element, for select smooth length "s"
       the smooth result of n th element is taking average from (n-s/2)-th to (n+s/2) th elements. """                                                                                    #create a 2d array
    s = int(smooth_pts/2)     
    w = len(input_data.transpose())
    l = len(input_data)
    smoothed_array = []
    for i in range(l):
        smoothed_array_col = []
        for j in range(w):
            if 0<= i < s:
                smoothed_pt = np.mean(input_data.iloc[0:2*i+1,j])

            elif s <= i <= l-s:
                smoothed_pt = np.mean(input_data.iloc[i-s:i+s,j])

            else:
                smoothed_pt = np.mean(input_data.iloc[i-(l-i)+1:l,j])
            smoothed_array_col.append(smoothed_pt)
        smoothed_array.append(smoothed_array_col)
    cutoff = 33356.68/(smooth_pts*t_step)
    print("The cutoff frequency =", round(cutoff,4), "cm-1")
    smoothed_result = pd.DataFrame(smoothed_array)
    
    #select the high-pass and low-pass filter
    if filtertype == "high_pass": 
        return input_data-smoothed_result
    
    elif filtertype == "low_pass":
        return smoothed_result
    
    else:# if not tpye,return the high-pass result 
        return input_data-smoothed_result
    

def add_window(input_data, zero_pt, t_step, start_pt, fft_number, window_function):
    """Apply the window function"""
    # transform the start and the end time
    start_ffttime = (start_pt-zero_pt)*t_step
    print("The selected start point of 2D FFT is ", start_ffttime, "fs")
    end_ffttime = (fft_number+start_pt-zero_pt)*t_step
    print("The selected end point of 2D FFT is ", end_ffttime, "fs")
    
    
    # Set the length of window function, not to over the length of input array
    if  start_pt+fft_number <= len(input_data):
        window_length = fft_number
    else:
        window_length = len(input_data)-start_pt
    
    window = signal.get_window(window_function, window_length)
    print("The selected window function is", window_function)
    
    windowed_data = (input_data.iloc[start_pt:start_pt+fft_number,:].transpose()*window/0.42).transpose()
    return pd.DataFrame(windowed_data)
    
    
def append0(input_data, t_step, apd0range_order):    
    """ add 0 into input data array to 2**n points """
    wavenumber_pitch = 33356/(t_step*(2**apd0range_order))
    print("The wavenumber pitch =", round(wavenumber_pitch, 4), "cm-1")
    pad_result = np.pad(input_data, ((0,2**apd0range_order-len(input_data)),(0,0)),"constant")
    return pd.DataFrame(pad_result)


def FFT_process(input_data, t_step, zero_pt, start_pt):
    """Input a 2D dA data and do the FFT among the time axis, and output the FFT result """ 
    # Get the number of sample points
    N = len(input_data)
    # Sample spacing
    T = t_step
    time_array = np.linspace(0,N*T,N,endpoint=False)
    FFT_result_2sides = fft(input_data.transpose()).transpose()
    
    # Take the one side FFT process (from 0 to the total length of input data)
    oneside_FFT_length = N/2+N%2   # Get the one side FFT length
    wavenumber_array = pd.Series(fftfreq(N,T)[:N//2]*N/2) # Get the wavenumber_array
    
    complexed_FFT_result0 = FFT_result_2sides[0:int(oneside_FFT_length),:] # The result of one side FFT 
    firstpt_reusLt0 = complexed_FFT_result0[0]/np.sqrt(2)
    complexed_FFT_result0[0] = firstpt_reusLt0
    complexed_FFT_result = complexed_FFT_result0*(np.sqrt(2)/complex(N,0))
    
    
    
    #display(complexed_FFT_result[:,50])
    #complexed_FFT_result = complexed_FFT_result0*(np.sqrt(2)/complex(length,0)
    real_result = (complexed_FFT_result).real
    imag_result = (complexed_FFT_result).imag
    
    # Power intensity of FFT
    mag_result = pd.DataFrame(np.sqrt(real_result**2+imag_result**2))

    # phase of FFT result
    phase_result = pd.DataFrame(np.arctan2(imag_result,real_result))
    #output_phase = []
    #for p in range(len(phase_result)):
    #    output_phase0 = ((np.pi/len(phase_result)*p*(zero_pt-start_pt))+phase_result.transpose()[p]+np.pi)%(2*np.pi)-np.pi
    #output_phase.append(output_phase0)    
    

    return wavenumber_array, complexed_FFT_result, mag_result, phase_result

### 1. dT transfer to dA

In [ ]:
dT_B_bg = bg(dT_b,20)
dT_F_bg = bg(dT_f,20)
dA_B_bg = -np.log10(1+dT_B_bg/T.loc[:,"X1"])
dA_F_bg = -np.log10(1+dT_F_bg/T.loc[:,"X1"])

### 2. dA transfer to dA-FFT

In [ ]:
zero_pt = 200
t_step = 0.99
start_pt = 400
fft_number = 1000
smooth_pts = 500

#### (1). Smooth the dA

In [ ]:
dA_B_bg_sm = smooth_filter(100000*dA_B_bg, t_step,smooth_pts, "high_pass")

#### (2). Add window

In [ ]:
dA_B_bg_sm_aw = add_window(dA_B_bg_sm, zero_pt, t_step, start_pt, fft_number, "blackman")

#### (3). Pad 0 to n order of 2

In [ ]:
dA_B_bg_sm_aw_pad0 = append0(dA_B_bg_sm_aw, t_step, 16)

In [ ]:
#FFT_wavenumber_array=np.linspace(0,0.5141*32768,32768,endpoint=False)
dA_FFT = FFT_process(dA_B_bg_sm_aw_pad0, 0.99,200,400)
wavenumber_array = dA_FFT[0]
dA_FFT_result = dA_FFT[2]

In [ ]:
fig1,ax1 = plt.subplots(constrained_layout=True, figsize=(16,9))       
curve1 = ax1.plot(wavenumber_array, np.abs(dA_FFT_result.iloc[:,67])**2, linewidth=5, color="black",label="Ferous Cyt " "$c$")
x_min, x_max=0, 3000
x_sticklength=5
x_sticknumber = int((x_max-x_min)/x_sticklength+1)
ax1.set_xlim(x_min,x_max)
y_min, y_max=0, 5
y_sticklength=1
y_sticknumber = int((y_max-y_min)/y_sticklength+1)
ax1.set_ylim(y_min,y_max)

In [ ]:
color_levels=np.linspace(0,2,101) 
fig1,ax1=plt.subplots(constrained_layout=True, figsize=(16,12))       
ax1.contourf(wavelength,wavenumber_array,dA_FFT_result,color_levels, cmap="jet",extend="both")
#=====================#
# Plot setting-x and y axis #
#=====================#
x_min, x_max=360, 440
x_sticklength=5
x_sticknumber=int((x_max-x_min)/x_sticklength+1)
ax1.set_xlim(x_min,x_max)
y_min,y_max=0,3000
y_sticklength=100
y_sticknumber=int((y_max-y_min)/y_sticklength+1)
ax1.set_ylim(y_min,y_max)

## Results plots
### 1.dA results of sample

In [ ]:
fig1,ax1 = plt.subplots(constrained_layout=True, figsize=(16,12))                                     # set the width and height of figure in 16/12 inchs
ax1.spines["top"].set_linewidth(2)
ax1.spines["left"].set_linewidth(2)
ax1.spines["bottom"].set_linewidth(2)
ax1.spines["right"].set_linewidth(2)                                                                  # set the linewidth of frame 

color_levels=np.linspace(-0.05,0.02,71)                                                               # set the levels of color map
#np.linspace(0.8*signal.min(),0.8*signal.max(),int((signal.max()-signal.min())/0.01))                                                                                                   
SP=ax1.contourf(wavelength, time_array, dA_B_bg, color_levels, cmap="jet",extend="both")
SP_line=plt.contour(wavelength, time_array, dA_B_bg,levels=(0,0.0015),linestyles="solid", colors="black",linewidths=0.5)
SP.cmap=plt.cm.get_cmap("jet").copy()
#SP.cmap.extendrect=True
SP.cmap.set_over("white")
SP.cmap.set_under("black")
plt.rc("csfont")
plt.gca

#====================#
# Plot setting-title #
#====================#
plt.title("$\Delta \it{A}$" " of DMSO-B Scan",fontdict={"family":"Times New Roman","size":40},pad=15)
plt.xlabel("Probe wavelngth (nm)",fontdict={"size":30})
plt.ylabel("Delay time (fs)",fontdict={"size":30})

#=====================#
# Plot setting-x axis #
#=====================#
x_min, x_max=360, 440
x_sticklength=5
x_sticknumber=int((x_max-x_min)/x_sticklength+1)
ax1.set_xlim(x_min,x_max)
x_wavelength=np.linspace(x_min,x_max,x_sticknumber,endpoint=True)
ax1.set_xticks(x_wavelength)
ax1.xaxis.set_tick_params(which="major",direction="in",top="on",length=10,width=1.5,pad=10,labelsize=18)
ax1.xaxis.get_ticklocs(minor=True)   
ax1.xaxis.set_minor_locator(MultipleLocator(1))
ax1.xaxis.set_tick_params(which="minor", bottom=True,top="on",direction="in",length=4,width=1.5)

#=====================#
# Plot setting-y axis #
#=====================#
y_min,y_max=-200,1800
y_sticklength=100
y_sticknumber=int((y_max-y_min)/y_sticklength+1)
ax1.set_ylim(y_min,y_max)
ax1.yaxis.set_ticks(np.linspace(y_min,y_max,y_sticknumber))  
ax1.yaxis.set_tick_params(direction="in",length=10,width=1.5,pad=10,labelsize=18)
ax1.yaxis.get_ticklocs(minor=True)   
ax1.yaxis.set_minor_locator(MultipleLocator(20))
ax1.yaxis.set_tick_params(which="minor", left=True,direction="in",length=4,width=1.5)

#===============================#
# Plot setting-secondary x axis #
#===============================#
x_photon_energy=(h*c/e)/(x_wavelength*10**(-9))
def photon_energy(wavelength):
    np.seterr(divide="ignore")
    return (h*c/e)/(wavelength*10**(-9))
def inverse(x2):
    return np.round(x2*(10**9)/(h*c/e),2)
secax=ax1.secondary_xaxis("top", functions=(photon_energy, photon_energy))
secax.set_xlabel("Photon energy(eV)",labelpad=15,fontdict={"size":30})
secax.set_xticks(x_photon_energy)
secax.tick_params(axis="x",which="major",direction="in",top="on",labeltop=True,length=10,width=1.5,pad=5,labelsize=18)
secax.xaxis.set_major_formatter(FormatStrFormatter("%.3f"))
#secax.minorticks_on
#secax.xaxis.set_minor_locator(MultipleLocator(5))
#secax.xaxis.set_tick_params(which='minor', bottom=True,direction="in",length=4,width=1.5)

#===============================#
# Plot setting-colcrbar setting #
#===============================#
cbar_min,cbar_max=-0.05, 0.02
cbar_level=15
cbar_ticks=np.linspace(cbar_min,                            # min of color bar
                       cbar_max,                            # max of color bar
                       cbar_level,                          
                       endpoint=True)           
cbar=fig1.colorbar(SP,ax=ax1,aspect=50,ticks=cbar_ticks,pad=-0.005,extendrect=True,extendfrac="auto")
#cbar.ax.set_title("Intensity",loc="left",pad=-0.1,fontdict={"size":15})
cbar.ax.tick_params(labelsize=15)
plt.savefig("fig1.jpg",dpi=300)
plt.show()

### 2. 2D FFT result of sample

In [ ]:
fig1,ax1 = plt.subplots(constrained_layout=True, figsize=(16,12))                                   # set the width and height of figure in 16/12 inchs
for arg in ("top", "bottom", "left", "right"):
    ax1.spines[arg].set_linewidth(2)

# ax1.spines["top"].set_linewidth(2)
# ax1.spines["left"].set_linewidth(2)
# ax1.spines["bottom"].set_linewidth(2)
# ax1.spines["right"].set_linewidth(2)                                                                  # set the linewidth of frame 

color_levels=np.linspace(0.05,2,100)                                                               # set the levels of color map
#np.linspace(0.8*signal.min(),0.8*signal.max(),int((signal.max()-signal.min())/0.01))                                                                                                   
SP=ax1.contourf(wavelength, wavenumber_array, np.abs(dA_FFT_result)**2, color_levels, cmap="jet",extend="both")
SP.cmap=plt.cm.get_cmap("jet").copy()
#SP.cmap.extendrect=True
SP.cmap.set_over("white")
SP.cmap.set_under("black")
plt.rc("csfont")
plt.gca

#====================#
# Plot setting-title #
#====================#
plt.title("$\Delta \it{A}$" " of DMSO-B Scan-FFT",fontdict={"family":"Times New Roman","size":40},pad=15)
plt.xlabel("Probe wavelngth (nm)",fontdict={"size":30})
plt.ylabel("Wavenumber (cm$^{-1}$)",fontdict={"size":30})

#=====================#
# Plot setting-x axis #
#=====================#
x_min, x_max=360, 440
x_sticklength=5
x_sticknumber=int((x_max-x_min)/x_sticklength+1)
ax1.set_xlim(x_min,x_max)
x_wavelength=np.linspace(x_min,x_max,x_sticknumber,endpoint=True)
ax1.set_xticks(x_wavelength)
ax1.xaxis.set_tick_params(which="major",direction="in",top="on",length=10,width=1.5,pad=10,labelsize=18)
ax1.xaxis.get_ticklocs(minor=True)   
ax1.xaxis.set_minor_locator(MultipleLocator(1))
ax1.xaxis.set_tick_params(which="minor", bottom=True,top="on",direction="in",length=4,width=1.5)

#=====================#
# Plot setting-y axis #
#=====================#
y_min,y_max=0,3000
y_sticklength=200
y_sticknumber=int((y_max-y_min)/y_sticklength+1)
ax1.set_ylim(y_min,y_max)
ax1.yaxis.set_ticks(np.linspace(y_min,y_max,y_sticknumber))  
ax1.yaxis.set_tick_params(direction="in",length=10,width=1.5,pad=10,labelsize=18)
ax1.yaxis.get_ticklocs(minor=True)   
ax1.yaxis.set_minor_locator(MultipleLocator(20))
ax1.yaxis.set_tick_params(which="minor", left=True,direction="in",length=4,width=1.5)

#===============================#
# Plot setting-secondary x axis #
#===============================#
x_photon_energy=(h*c/e)/(x_wavelength*10**(-9))

def photon_energy(wavelength):
    np.seterr(divide="ignore")
    return (h*c/e)/(wavelength*10**(-9))

def inverse(x2):
    return np.round(x2*(10**9)/(h*c/e),2)

secax = ax1.secondary_xaxis("top", functions=(photon_energy, photon_energy))
secax.set_xlabel("Photon energy(eV)", labelpad=15, fontdict={"size": 30})
secax.set_xticks(x_photon_energy)
secax.tick_params(axis="x", which="major", direction="in", top="on", labeltop=True,
                  length=10, width=1.5, pad=5, labelsize=18)
secax.xaxis.set_major_formatter(FormatStrFormatter("%.3f"))
#secax.minorticks_on
#secax.xaxis.set_minor_locator(MultipleLocator(5))
#secax.xaxis.set_tick_params(which='minor', bottom=True,direction="in",length=4,width=1.5)

#===============================#
# Plot setting-colcrbar setting #
#===============================#
cbar_min, cbar_max = 0.05, 2
cbar_level = 15
cbar_ticks = np.linspace(cbar_min,                            # min of color bar
                       cbar_max,                            # max of color bar
                       cbar_level,                          
                       endpoint=True)           
cbar = fig1.colorbar(SP,ax=ax1,aspect=50,ticks=cbar_ticks,pad=-0.005,extendrect=True,extendfrac="auto")
#cbar.ax.set_title("Intensity",loc="left",pad=-0.1,fontdict={"size":15})
cbar.ax.tick_params(labelsize=15)
plt.savefig("fig2.jpg",dpi=300)
plt.show()

In [ ]:
test= pd.DataFrame([[1 for i in range(96)] for m in range(2001)])
test_aw = add_window(test, 0, 0.99, 0, 2001, "blackman")
test_ap0 = append0(test_aw, 0.99, 16)
test_fft = FFT_process(test_ap0 ,0.99,0,0)
#display(pd.DataFrame(test_fft[0]))

In [ ]:
fig1,ax1 = plt.subplots(constrained_layout=True, figsize=(16,12))       
curve1 = ax1.plot(test_fft[0], test_fft[2].iloc[:,50], linewidth=5, color="black",label="Ferous Cyt " "$c$")
x_min, x_max=0, 100
x_sticklength=5
x_sticknumber = int((x_max-x_min)/x_sticklength+1)
ax1.set_xlim(x_min,x_max)
y_min, y_max=0, 0.05
y_sticklength=1
y_sticknumber = int((y_max-y_min)/y_sticklength+1)
ax1.set_ylim(y_min,y_max)